In [1]:
import tensorflow as tf  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import keras

In [3]:
import glob

In [12]:
train_files=glob.glob("C:/Users/HP/Dropbox/PC (2)/Downloads/archive (1)/flowers*/*/**")

In [13]:
from random import shuffle
shuffle( train_files)
train_files

['C:/Users/HP/Dropbox/PC (2)/Downloads/archive (1)\\flowers\\7\\3814298044_93fe954be3_c.jpg',
 'C:/Users/HP/Dropbox/PC (2)/Downloads/archive (1)\\flowers\\D\\6165833664_dc722e3e0f_c.jpg',
 'C:/Users/HP/Dropbox/PC (2)/Downloads/archive (1)\\flowers\\E\\8480457565_18d0098c0f_c.jpg',
 'C:/Users/HP/Dropbox/PC (2)/Downloads/archive (1)\\flowers\\A\\3540145999_df7c5087ca_c.jpg',
 'C:/Users/HP/Dropbox/PC (2)/Downloads/archive (1)\\flowers\\B\\51002680270_c5e9851a93_c.jpg',
 'C:/Users/HP/Dropbox/PC (2)/Downloads/archive (1)\\flowers\\E\\24806304776_05b479a165_c.jpg',
 'C:/Users/HP/Dropbox/PC (2)/Downloads/archive (1)\\flowers\\B\\8623980368_b5001cd781_c.jpg',
 'C:/Users/HP/Dropbox/PC (2)/Downloads/archive (1)\\flowers\\3\\30127978573_dd3cd7fe88_c.jpg',
 'C:/Users/HP/Dropbox/PC (2)/Downloads/archive (1)\\flowers\\8\\1981808_9421e9301a_c.jpg',
 'C:/Users/HP/Dropbox/PC (2)/Downloads/archive (1)\\flowers\\A\\4594894734_4404a6ea9b_c.jpg',
 'C:/Users/HP/Dropbox/PC (2)/Downloads/archive (1)\\flowers\

In [14]:
def flower_class(fc):
    if fc.find("astilbe") != -1 :
        return 1
    elif fc.find("bellflower") != -1 :
        return 2
    elif fc.find("black_eyed_susan") != -1 :
        return 3
    elif fc.find("calendula") != -1 :
        return 4
    elif fc.find("california_poppy") != -1 :
        return 5
    elif fc.find("carnation") != -1 :
        return 6
    elif fc.find("common_daisy") != -1 :
        return 7
    elif fc.find("coreopsis") != -1 :
        return 8
    elif fc.find("daffodil") != -1 :
        return 9
    elif fc.find("iris") != -1 :
        return 10
    elif fc.find("magnolia") != -1 :
        return 11
    elif fc.find("rose") != -1 :
        return 12
    elif fc.find("sunflower") != -1 :
        return 13
    elif fc.find("tulip") != -1 :
        return 14
    else:
        return 0

In [15]:
from PIL import Image

In [18]:
X_train=np.zeros((len(train_files),64,64,3))
Y_train=np.zeros((len(train_files)))

for i,fc in enumerate(train_files):
    img = Image.open(fc)
    img = img.resize((64,64))
    arr = np.array(img)
    
    arr=arr/255
    
    if arr.shape == (64,64,3):
        X_train[i,:,:,:]=arr   
    d = fc[57] 
    
    if (d>='A' and d < 'Z'):
            if (d=='A') :
                Y_train[i] = 10
            elif (d=='B') :
                Y_train[i] = 11
            elif (d=='C'):
                Y_train[i] = 12
            elif (d=='D') :
                Y_train[i] = 13
            elif (d=='E') :
                Y_train[i] = 14
            elif (d=='F') :Y_train[i] = 15
    else :
            Y_train[i] = d

In [19]:
from matplotlib import pyplot as plt

In [20]:
np.count_nonzero(Y_train == 0)

737

In [21]:
from keras import layers, Sequential

In [22]:
from keras.layers import Conv2D, Dense, Flatten

In [23]:
import keras_tuner

In [24]:
from keras_tuner import engine
from keras_tuner.engine.hyperparameters import HyperParameters as hp

In [25]:
def build_model(hp):  
  model = keras.Sequential([
      keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),#min value of filter you want
        kernel_size=hp.Choice('conv_1_kernel', values = [2,4]),#picks like mcq
        activation='relu',
        input_shape=(64,64,3)
        
    ),
    keras.layers.Dropout((0.5)),
      #for i in range(0,hp.Int(....)) this will also work
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),# ihave added this line extra in comparison to the old model
        kernel_size=hp.Choice('conv_2_kernel', values = [2,3]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
   # keras.layers.Dense(28, activation='sigmoid'),
   # keras.layers.Dense(28, activation='relu'),
    
    keras.layers.Dense(16, activation='softmax')
  ])
    
  
  model.compile(optimizer=keras.optimizers.Adam(learning_rate = 1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [ ]:
tuner_search=keras_tuner.RandomSearch(
    build_model,
    objective='accuracy',
    max_trials=6,
    directory='output1',
)

In [27]:
tuner_search.search(X_train, Y_train, epochs=2, validation_split=0.2)

Trial 6 Complete [00h 03m 31s]
accuracy: 0.4153430759906769

Best accuracy So Far: 0.5177890658378601
Total elapsed time: 00h 56m 54s
INFO:tensorflow:Oracle triggered exit


In [28]:
model = tuner_search.get_best_models(num_models=1)[0]

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 63, 63, 96)        1248      
                                                                 
 dropout (Dropout)           (None, 63, 63, 96)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 62, 64)        24640     
                                                                 
 flatten (Flatten)           (None, 246016)            0         
                                                                 
 dense (Dense)               (None, 80)                19681360  
                                                                 
 dense_1 (Dense)             (None, 16)                1296      
                                                                 
Total params: 19,708,544
Trainable params: 19,708,544
No

In [30]:
model.fit(X_train, Y_train, epochs=5, validation_split=0.2, initial_epoch=2)

Epoch 3/5
394/394 [==============================] - 307s 779ms/step - loss: 1.0348 - accuracy: 0.6589 - val_loss: 1.5071 - val_accuracy: 0.5162
Epoch 4/5
114/394 [=======>......................] - ETA: 3:30 - loss: 0.6291 - accuracy: 0.8032

KeyboardInterrupt: 

In [ ]:
predictions=model.predict(X_check)

In [ ]:
predictions

In [ ]:
out=np.argmax(predictions,axis=1)

In [ ]:
out

In [ ]:
def flower_name(x):
    
    if x==1:
        return "astilbe"
    elif x==2:
        return "bellflower"
    elif x==7:
        return "common daisy"
    elif x==9:
        return "daffodil"
    elif x==13:
        return "rose"
    elif x==14:
        return "tulip"
    elif x==0:
        return "water lily"
    else:
        return "some other flower"